In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ufc-fight-statistics-july-2016-nov-2024/UFC Fight Statistics Disparities (July 2016 - Nov 2024).csv
/kaggle/input/ufc-fight-statistics-july-2016-nov-2024/UFC_Fights_5Rd_Decisions_Disparities.csv
/kaggle/input/ufc-fight-statistics-july-2016-nov-2024/UFC Fight Statistics (July 2016 - Nov 2024).csv
/kaggle/input/ufc-fight-statistics-july-2016-nov-2024/UFC_Stats_Scraper.ipynb
/kaggle/input/ufc-fight-statistics-july-2016-nov-2024/UFC_Fights_3Rd_Decisions_Disparities.csv


In [2]:
from xgboost import XGBClassifier
from sklearn.metrics import log_loss, brier_score_loss, roc_auc_score

import warnings
warnings.filterwarnings("ignore")

# Loading disparities dataset
path = "/kaggle/input/ufc-fight-statistics-july-2016-nov-2024/UFC Fight Statistics Disparities (July 2016 - Nov 2024).csv"
df = pd.read_csv(path)

print(df.shape)
df.head()

(4148, 105)


,Fighter1,Fighter2,F1 Winner?,F2 Winner?,Fight Method,Round,Time,Time Format,Referee,Finish Details or Judges Scorecard,...,Leg R5 Disp.,Distance R5 Disp.,Clinch R5 Disp.,Ground R5 Disp.,Knockdowns R5 Disp.,TD Completed R5 Disp.,TD Missed R5 Disp.,Sub. Att R5 Disp.,Rev. R5 Disp.,Ctrl Time (Minutes) R5 Disp.
0,Brandon Moreno,Amir Albazi,1,0,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Marc Goddard,Eric Colon 46 - 49. Sal D'amato 45 - 50. Derek...,...,0.0,22.0,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,-0.116667
1,Erin Blanchfield,Rose Namajunas,1,0,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Blake Grice,Junichiro Kamijo 47 - 48. Mike Bell 47 - 48. D...,...,0.0,2.0,2.0,4.0,0.0,1.0,0.0,0.0,0.0,4.133333
2,Caio Machado,Brendson Ribeiro,0,1,Decision - Split,3,5:00,3 Rnd (5-5-5),Jerin Valel,Sal D'amato 28 - 29. Thomas Collins 29 - 28. M...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ariane da Silva,Jasmine Jasudavicius,0,1,Submission,3,2:28,3 Rnd (5-5-5),Luke Boutin,D'Arce Choke On Ground,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Marc-Andre Barriault,Dustin Stoltzfus,0,1,KO/TKO,1,4:28,3 Rnd (5-5-5),Marc Goddard,Punch to Head In Clinch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
[x for x in df.columns if "Disp" in x]


['Total Strike Landed R1  Disp.',
 'Total Strike Missed R1  Disp.',
 'Non-Sig. Strike Landed R1  Disp.',
 'Non-Sig. Strike Missed R1  Disp.',
 'Sig. Strike Landed R1  Disp.',
 'Sig. Strike Missed R1  Disp.',
 'Head R1  Disp.',
 'Body R1  Disp.',
 'Leg R1  Disp.',
 'Distance R1  Disp.',
 'Clinch R1  Disp.',
 'Ground R1  Disp.',
 'Knockdowns R1  Disp.',
 'TD Completed R1  Disp.',
 'TD Missed R1  Disp.',
 'Sub. Att R1  Disp.',
 'Rev. R1  Disp.',
 'Ctrl Time (Minutes) R1  Disp.',
 'Total Strike Landed R2  Disp.',
 'Total Strike Missed R2  Disp.',
 'Non-Sig. Strike Landed R2  Disp.',
 'Non-Sig. Strike Missed R2  Disp.',
 'Sig. Strike Landed R2  Disp.',
 'Sig. Strike Missed R2  Disp.',
 'Head R2  Disp.',
 'Body R2  Disp.',
 'Leg R2  Disp.',
 'Distance R2  Disp.',
 'Clinch R2  Disp.',
 'Ground R2  Disp.',
 'Knockdowns R2  Disp.',
 'TD Completed R2  Disp.',
 'TD Missed R2  Disp.',
 'Sub. Att R2  Disp.',
 'Rev. R2  Disp.',
 'Ctrl Time (Minutes) R2  Disp.',
 'Total Strike Landed R3  Disp.',
 'To

In [4]:
df['Date'] = pd.to_datetime(df['Date'])
df['f1_win'] = df['F1 Winner?'].astype(int)
print(df.shape)

(4148, 106)


Building per-fighter performance table (2 rows per fight)

In [5]:
# 1) all disparity columns (these are F1 - F2 for that round/stat)
disp_cols = [c for c in df.columns if "Disp." in c]

# ensure numeric
for c in disp_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 2) Fighter1 perspective
f1_perf = df[['Date', 'Fighter1', 'Fighter2', 'F1 Winner?'] + disp_cols].copy()
f1_perf = f1_perf.rename(columns={
    'Fighter1': 'fighter',
    'Fighter2': 'opponent',
    'F1 Winner?': 'win'
})

# 3) Fighter2 perspective (flip sign on all disparity stats)
f2_perf = df[['Date', 'Fighter1', 'Fighter2', 'F2 Winner?'] + disp_cols].copy()
f2_perf = f2_perf.rename(columns={
    'Fighter2': 'fighter',
    'Fighter1': 'opponent',
    'F2 Winner?': 'win'
})

for c in disp_cols:
    # from F2 viewpoint, differential is the opposite
    f2_perf[c] = -f2_perf[c]

# 4) Stack both perspectives
fighter_perf = pd.concat([f1_perf, f2_perf], ignore_index=True)
fighter_perf = fighter_perf.sort_values(['fighter', 'Date']).reset_index(drop=True)

fighter_perf.head()


,Date,fighter,opponent,win,Total Strike Landed R1 Disp.,Total Strike Missed R1 Disp.,Non-Sig. Strike Landed R1 Disp.,Non-Sig. Strike Missed R1 Disp.,Sig. Strike Landed R1 Disp.,Sig. Strike Missed R1 Disp.,...,Leg R5 Disp.,Distance R5 Disp.,Clinch R5 Disp.,Ground R5 Disp.,Knockdowns R5 Disp.,TD Completed R5 Disp.,TD Missed R5 Disp.,Sub. Att R5 Disp.,Rev. R5 Disp.,Ctrl Time (Minutes) R5 Disp.
0,2024-03-02,AJ Cunningham,Ludovit Klein,0,-27,9,-3,1,-24,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-12,AJ Dobson,Jacob Malkoun,0,20,5,3,1,17,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-22,AJ Dobson,Armen Petrosyan,0,-18,2,2,0,-20,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-08-12,AJ Dobson,Tafon Nchukwi,1,12,12,2,2,10,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-03-23,AJ Dobson,Edmen Shahbazyan,0,-11,14,1,-1,-12,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Building pre-fight cumulative stats

In [6]:
# columns to average
perf_cols = ['win'] + disp_cols

def compute_pre_profiles(group):
    group = group.sort_values('Date')
    for col in perf_cols:
        group[f'pre_{col}_mean'] = group[col].expanding().mean().shift(1)
    return group

fighter_prof = fighter_perf.groupby('fighter', group_keys=False).apply(compute_pre_profiles)
fighter_prof.head()

,Date,fighter,opponent,win,Total Strike Landed R1 Disp.,Total Strike Missed R1 Disp.,Non-Sig. Strike Landed R1 Disp.,Non-Sig. Strike Missed R1 Disp.,Sig. Strike Landed R1 Disp.,Sig. Strike Missed R1 Disp.,...,pre_Leg R5 Disp._mean,pre_Distance R5 Disp._mean,pre_Clinch R5 Disp._mean,pre_Ground R5 Disp._mean,pre_Knockdowns R5 Disp._mean,pre_TD Completed R5 Disp._mean,pre_TD Missed R5 Disp._mean,pre_Sub. Att R5 Disp._mean,pre_Rev. R5 Disp._mean,pre_Ctrl Time (Minutes) R5 Disp._mean
0,2024-03-02,AJ Cunningham,Ludovit Klein,0,-27,9,-3,1,-24,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-12,AJ Dobson,Jacob Malkoun,0,20,5,3,1,17,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-22,AJ Dobson,Armen Petrosyan,0,-18,2,2,0,-20,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-08-12,AJ Dobson,Tafon Nchukwi,1,12,12,2,2,10,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-03-23,AJ Dobson,Edmen Shahbazyan,0,-11,14,1,-1,-12,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_fight = df.copy().reset_index().rename(columns={'index': 'fight_id'})

pre_cols = [c for c in fighter_prof.columns if c.startswith('pre_')]

# F1 pre-fight profile
f1_pre = fighter_prof[['fighter', 'Date'] + pre_cols].copy()
f1_pre = f1_pre.rename(columns={'fighter': 'Fighter1'})
df_merged = pd.merge(df_fight, f1_pre,
                     how='left',
                     on=['Fighter1', 'Date'],
                     suffixes=('', '_F1pre'))

# F2 pre-fight profile
f2_pre = fighter_prof[['fighter', 'Date'] + pre_cols].copy()
f2_pre = f2_pre.rename(columns={'fighter': 'Fighter2'})
df_merged = pd.merge(df_merged, f2_pre,
                     how='left',
                     on=['Fighter2', 'Date'],
                     suffixes=('', '_F2pre'))

df_merged.shape


(4148, 289)

In [8]:
pre_cols_f1 = [c for c in df_merged.columns
               if c.startswith('pre_') and not c.endswith('_F2pre')]

feature_cols = []

for c in pre_cols_f1:
    c_f1 = c
    c_f2 = c + '_F2pre'
    if c_f2 in df_merged.columns:
        new_name = 'diff_' + c[4:]  # strip 'pre_'
        df_merged[new_name] = df_merged[c_f1] - df_merged[c_f2]
        feature_cols.append(new_name)

len(feature_cols), feature_cols[:10]


(91,
 ['diff_win_mean',
  'diff_Total Strike Landed R1  Disp._mean',
  'diff_Total Strike Missed R1  Disp._mean',
  'diff_Non-Sig. Strike Landed R1  Disp._mean',
  'diff_Non-Sig. Strike Missed R1  Disp._mean',
  'diff_Sig. Strike Landed R1  Disp._mean',
  'diff_Sig. Strike Missed R1  Disp._mean',
  'diff_Head R1  Disp._mean',
  'diff_Body R1  Disp._mean',
  'diff_Leg R1  Disp._mean'])

In [9]:
model_df = df_merged.dropna(subset=feature_cols + ['f1_win', 'Date']).copy()
model_df = model_df.sort_values('Date')

X = model_df[feature_cols]
y = model_df['f1_win'].astype(int)
X.shape, y.shape


((185, 91), (185,))

Train/valid split and XGBoost

In [10]:
split_idx = int(0.8 * len(model_df))

X_train = X.iloc[:split_idx]
y_train = y.iloc[:split_idx]
X_valid = X.iloc[split_idx:]
y_valid = y.iloc[split_idx:]

model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    eval_metric='logloss',
    tree_method='hist',
    random_state=42
)

model.fit(X_train, y_train)

p_train = model.predict_proba(X_train)[:, 1]
p_valid = model.predict_proba(X_valid)[:, 1]

print("Train logloss:", log_loss(y_train, p_train))
print("Valid logloss:", log_loss(y_valid, p_valid))
print("Valid Brier:", brier_score_loss(y_valid, p_valid))
print("Valid AUC:", roc_auc_score(y_valid, p_valid))


Train logloss: 0.030683380848315584
Valid logloss: 1.166507133580856
Valid Brier: 0.37974477151032976
Valid AUC: 0.22121212121212122


In [11]:
disp_cols = [c for c in df.columns if "Disp." in c]

sig_str_cols = [c for c in disp_cols if "Sig. Strike Landed" in c]
tot_str_cols = [c for c in disp_cols if "Total Strike Landed" in c]
kd_cols      = [c for c in disp_cols if "Knockdowns" in c]
td_cols      = [c for c in disp_cols if "TD Completed" in c]
ctrl_cols    = [c for c in disp_cols if "Ctrl Time" in c]
sub_cols     = [c for c in disp_cols if "Sub. Att" in c]


In [12]:
df['SigStr_diff'] = df[sig_str_cols].sum(axis=1)
df['TotStr_diff'] = df[tot_str_cols].sum(axis=1)
df['KD_diff']     = df[kd_cols].sum(axis=1)
df['TD_diff']     = df[td_cols].sum(axis=1)
df['Ctrl_diff']   = df[ctrl_cols].sum(axis=1)
df['Sub_diff']    = df[sub_cols].sum(axis=1)


In [13]:
stats = ['SigStr_diff','TotStr_diff','KD_diff','TD_diff','Ctrl_diff','Sub_diff']


In [14]:
# Fighter 1 performance (already F1-F2)
f1_perf = df[['Date','Fighter1','Fighter2','f1_win'] + stats].copy()
f1_perf = f1_perf.rename(columns={
    'Fighter1':'fighter',
    'Fighter2':'opponent',
    'f1_win':'win'
})

# Fighter 2 performance → flip sign
f2_perf = df[['Date','Fighter1','Fighter2','f1_win'] + stats].copy()
f2_perf['win'] = 1 - f2_perf['f1_win']
f2_perf = f2_perf.rename(columns={
    'Fighter2':'fighter',
    'Fighter1':'opponent'
})

# flip differential signs (F2 POV = negative)
for s in stats:
    f2_perf[s] = -f2_perf[s]

fighter_perf = pd.concat([f1_perf, f2_perf], ignore_index=True)
fighter_perf = fighter_perf.sort_values(["fighter","Date"]).reset_index(drop=True)
fighter_perf.head()


,Date,fighter,opponent,win,SigStr_diff,TotStr_diff,KD_diff,TD_diff,Ctrl_diff,Sub_diff,f1_win
0,2024-03-02,AJ Cunningham,Ludovit Klein,0,-27.0,-27.0,-1.0,-2.0,-0.016667,-0.0,1.0
1,2022-02-12,AJ Dobson,Jacob Malkoun,0,-115.0,-115.0,0.0,-6.0,-8.833333,0.0,NaN
2,2022-10-22,AJ Dobson,Armen Petrosyan,0,-54.0,-54.0,-0.0,3.0,0.816667,-0.0,1.0
3,2023-08-12,AJ Dobson,Tafon Nchukwi,1,39.0,39.0,0.0,2.0,4.550000,0.0,NaN
4,2024-03-23,AJ Dobson,Edmen Shahbazyan,0,-11.0,-11.0,-1.0,-1.0,0.433333,-0.0,1.0


In [15]:
perf_cols = ['win'] + stats

def compute_past(group):
    group = group.sort_values('Date')
    for col in perf_cols:
        group[f'pre_{col}_mean'] = group[col].expanding().mean().shift(1)
    return group

fighter_prof = fighter_perf.groupby('fighter', group_keys=False).apply(compute_past)
fighter_prof.head()


,Date,fighter,opponent,win,SigStr_diff,TotStr_diff,KD_diff,TD_diff,Ctrl_diff,Sub_diff,f1_win,pre_win_mean,pre_SigStr_diff_mean,pre_TotStr_diff_mean,pre_KD_diff_mean,pre_TD_diff_mean,pre_Ctrl_diff_mean,pre_Sub_diff_mean
0,2024-03-02,AJ Cunningham,Ludovit Klein,0,-27.0,-27.0,-1.0,-2.0,-0.016667,-0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-12,AJ Dobson,Jacob Malkoun,0,-115.0,-115.0,0.0,-6.0,-8.833333,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-22,AJ Dobson,Armen Petrosyan,0,-54.0,-54.0,-0.0,3.0,0.816667,-0.0,1.0,0.000000,-115.000000,-115.000000,0.0,-6.000000,-8.833333,0.0
3,2023-08-12,AJ Dobson,Tafon Nchukwi,1,39.0,39.0,0.0,2.0,4.550000,0.0,NaN,0.000000,-84.500000,-84.500000,-0.0,-1.500000,-4.008333,-0.0
4,2024-03-23,AJ Dobson,Edmen Shahbazyan,0,-11.0,-11.0,-1.0,-1.0,0.433333,-0.0,1.0,0.333333,-43.333333,-43.333333,0.0,-0.333333,-1.155556,0.0


In [16]:
df_fight = df.copy().reset_index().rename(columns={'index':'fight_id'})

pre_cols = [c for c in fighter_prof.columns if c.startswith("pre_")]

# F1 merge
f1_pre = fighter_prof[['fighter','Date'] + pre_cols].rename(columns={'fighter':'Fighter1'})
dfm = df_fight.merge(f1_pre, how='left', on=['Fighter1','Date'], suffixes=('', '_F1pre'))

# F2 merge
f2_pre = fighter_prof[['fighter','Date'] + pre_cols].rename(columns={'fighter':'Fighter2'})
dfm = dfm.merge(f2_pre, how='left', on=['Fighter2','Date'], suffixes=('', '_F2pre'))


In [17]:
feature_cols = []

for c in pre_cols:
    c_f1 = c
    c_f2 = c + "_F2pre"
    if c_f2 in dfm.columns:
        new = "diff_" + c[4:]        # skip "pre_"
        dfm[new] = dfm[c_f1] - dfm[c_f2]
        feature_cols.append(new)

model_df = dfm.dropna(subset=feature_cols + ['f1_win']).copy()
model_df = model_df.sort_values('Date')
X = model_df[feature_cols]
y = model_df['f1_win'].astype(int)


In [18]:
split_idx = int(0.8 * len(model_df))

X_train = X.iloc[:split_idx]
y_train = y.iloc[:split_idx]
X_valid = X.iloc[split_idx:]
y_valid = y.iloc[split_idx:]

model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    eval_metric='logloss',
    tree_method='hist',
    random_state=42
)

model.fit(X_train, y_train)

p_train = model.predict_proba(X_train)[:,1]
p_valid = model.predict_proba(X_valid)[:,1]

print("Train logloss:", log_loss(y_train, p_train))
print("Valid logloss:", log_loss(y_valid, p_valid))
print("Valid Brier:",  brier_score_loss(y_valid, p_valid))
print("Valid AUC:",    roc_auc_score(y_valid, p_valid))


Train logloss: 0.4313650150631419
Valid logloss: 0.7040606502125583
Valid Brier: 0.25077857369241485
Valid AUC: 0.5736525999683895


Getting latest pre-fight profile for a fighter

In [19]:
def get_latest_pre_profile(fighter_name):
    sub = fighter_prof[fighter_prof['fighter'] == fighter_name]
    # need at least one row with non-NaN profile
    sub = sub.dropna(subset=['pre_win_mean'])
    if sub.empty:
        return None
    # last fight chronologically
    return sub.sort_values('Date').iloc[-1]


feature vector for a matchup

In [20]:
def build_features_for_matchup(f1_name, f2_name, pre_cols, feature_cols):
    r1 = get_latest_pre_profile(f1_name)
    r2 = get_latest_pre_profile(f2_name)
    if r1 is None or r2 is None:
        print(f"No history for {f1_name} or {f2_name}")
        return None
    
    feat = {}
    for c in pre_cols:
        base = c[4:]           # strip "pre_"
        feat_name = 'diff_' + base
        if feat_name in feature_cols:
            feat[feat_name] = r1[c] - r2[c]
    
    # ensure columns in same order as training
    feat_vec = pd.DataFrame([feat])[feature_cols]
    return feat_vec


today’s card + odds

In [21]:
today_fights = [
    ("Arman Tsarukyan", "Dan Hooker"),
    ("Ian Machado Garry", "Belal Muhammad"),
    ("Volkan Oezdemir", "Alonzo Menifield"),
    ("Mykytybek Orolbai", "Jack Hermansson"),
    ("Waldo Cortes-Acosta", "Shamil Gaziev"),
    ("Tagir Ulanbekov", "Kyoji Horiguchi"),
]

amer_odds = {
    ("Arman Tsarukyan", "Dan Hooker"):        (-650,  470),
    ("Ian Machado Garry", "Belal Muhammad"):  (-270,  220),
    ("Volkan Oezdemir", "Alonzo Menifield"):  (-225,  185),
    ("Mykytybek Orolbai", "Jack Hermansson"): (-250,  205),
    ("Waldo Cortes-Acosta", "Shamil Gaziev"): (-135,  114),
    ("Tagir Ulanbekov", "Kyoji Horiguchi"):   (-218,  180),
}


Odds converters + EV function

In [22]:
def american_to_decimal(o):
    if o > 0:
        return 1 + o / 100.0
    else:
        return 1 + 100.0 / (-o)

def expected_value(p_win, dec_odds, stake=1.0):
    # profit if win = (dec_odds - 1)*stake
    # loss if lose = stake
    return p_win * (dec_odds - 1.0) * stake - (1 - p_win) * stake


model probabilities and EV for each fight

In [23]:
results = []

for f1, f2 in today_fights:
    feat_vec = build_features_for_matchup(f1, f2, pre_cols, feature_cols)
    if feat_vec is None:
        continue
    
    p_f1 = model.predict_proba(feat_vec)[:, 1][0]
    p_f2 = 1 - p_f1
    
    o1, o2 = amer_odds[(f1, f2)]
    d1 = american_to_decimal(o1)
    d2 = american_to_decimal(o2)
    
    ev1 = expected_value(p_f1, d1)
    ev2 = expected_value(p_f2, d2)
    
    results.append({
        "F1": f1,
        "F2": f2,
        "p_F1_model": p_f1,
        "p_F2_model": p_f2,
        "F1_american": o1,
        "F2_american": o2,
        "F1_decimal": d1,
        "F2_decimal": d2,
        "EV_F1": ev1,
        "EV_F2": ev2,
    })

results_df = pd.DataFrame(results)
results_df


No history for Mykytybek Orolbai or Jack Hermansson
No history for Tagir Ulanbekov or Kyoji Horiguchi


,F1,F2,p_F1_model,p_F2_model,F1_american,F2_american,F1_decimal,F2_decimal,EV_F1,EV_F2
0,Arman Tsarukyan,Dan Hooker,0.766113,0.233887,-650,470,1.153846,5.70,-0.116024,0.333158
1,Ian Machado Garry,Belal Muhammad,0.715297,0.284703,-270,220,1.370370,3.20,-0.019778,-0.088951
2,Volkan Oezdemir,Alonzo Menifield,0.477281,0.522719,-225,185,1.444444,2.85,-0.310594,0.489750
3,Waldo Cortes-Acosta,Shamil Gaziev,0.749607,0.250393,-135,114,1.740741,2.14,0.304871,-0.464158


In [24]:
# Sort by best EV on favourites
results_df.sort_values("EV_F1", ascending=False)

# Sort by best EV on underdogs
results_df.sort_values("EV_F2", ascending=False)


,F1,F2,p_F1_model,p_F2_model,F1_american,F2_american,F1_decimal,F2_decimal,EV_F1,EV_F2
2,Volkan Oezdemir,Alonzo Menifield,0.477281,0.522719,-225,185,1.444444,2.85,-0.310594,0.489750
0,Arman Tsarukyan,Dan Hooker,0.766113,0.233887,-650,470,1.153846,5.70,-0.116024,0.333158
1,Ian Machado Garry,Belal Muhammad,0.715297,0.284703,-270,220,1.370370,3.20,-0.019778,-0.088951
3,Waldo Cortes-Acosta,Shamil Gaziev,0.749607,0.250393,-135,114,1.740741,2.14,0.304871,-0.464158


In [25]:
perf_cols = ['win'] + stats  # stats = ['SigStr_diff','TotStr_diff','KD_diff','TD_diff','Ctrl_diff','Sub_diff']

def compute_profiles_recency(group, alpha=0.3):
    group = group.sort_values('Date')
    for col in perf_cols:
        # old simple expanding mean (baseline)
        group[f'pre_{col}_mean'] = group[col].expanding().mean().shift(1)
        # new recency-weighted exponential moving average
        group[f'pre_{col}_ema'] = group[col].ewm(alpha=alpha, adjust=False).mean().shift(1)
    return group

fighter_prof = fighter_perf.groupby('fighter', group_keys=False).apply(compute_profiles_recency)


In [26]:
pre_cols = [c for c in fighter_prof.columns if c.startswith('pre_')]


Elo rating to each fight

In [27]:
import math
from collections import defaultdict

START_RATING = 1500
K = 32

elo = defaultdict(lambda: START_RATING)

elo_f1_list = []
elo_f2_list = []

df_sorted = df.sort_values('Date').copy()

for _, row in df_sorted.iterrows():
    f1 = row['Fighter1']
    f2 = row['Fighter2']
    result = row['f1_win']   # 1 if F1 wins, 0 if F2 wins
    
    r1 = elo[f1]
    r2 = elo[f2]
    
    # expected scores
    exp1 = 1 / (1 + 10 ** ((r2 - r1) / 400))
    exp2 = 1 - exp1
    
    s1 = result
    s2 = 1 - result
    
    # store pre-fight ratings
    elo_f1_list.append(r1)
    elo_f2_list.append(r2)
    
    # update ratings after the fight
    elo[f1] = r1 + K * (s1 - exp1)
    elo[f2] = r2 + K * (s2 - exp2)

# attach back to df in original row order
df_sorted['elo_F1'] = elo_f1_list
df_sorted['elo_F2'] = elo_f2_list
df = df_sorted.sort_index()

df['elo_diff'] = df['elo_F1'] - df['elo_F2']


Rebuilding fight-level merged frame with mean + EMA + Elo

In [28]:
# rebuild fight-level base with elo_diff already attached
df_fight = df.copy().reset_index().rename(columns={'index': 'fight_id'})

# merge pre-fight profiles for Fighter1
f1_pre = fighter_prof[['fighter', 'Date'] + pre_cols].rename(columns={'fighter': 'Fighter1'})
dfm = df_fight.merge(f1_pre, on=['Fighter1', 'Date'], how='left')

# merge pre-fight profiles for Fighter2
f2_pre = fighter_prof[['fighter', 'Date'] + pre_cols].rename(columns={'fighter': 'Fighter2'})
dfm = dfm.merge(f2_pre, on=['Fighter2', 'Date'], how='left', suffixes=('', '_F2pre'))


In [29]:
pre_cols_f1 = [c for c in dfm.columns 
               if c.startswith('pre_') and not c.endswith('_F2pre')]

feature_cols = []

for c in pre_cols_f1:
    c_f1 = c
    c_f2 = c + '_F2pre'
    if c_f2 in dfm.columns:
        name = 'diff_' + c[4:]   # strip 'pre_'
        dfm[name] = dfm[c_f1] - dfm[c_f2]
        feature_cols.append(name)

# add Elo rating difference as an extra feature
feature_cols.append('elo_diff')

model_df = dfm.dropna(subset=feature_cols + ['f1_win']).copy()
model_df = model_df.sort_values('Date')

X = model_df[feature_cols]
y = model_df['f1_win'].astype(int)
X.shape, y.shape


((3004, 15), (3004,))

Retraining XGBoost with upgraded features

In [30]:
from xgboost import XGBClassifier
from sklearn.metrics import log_loss, brier_score_loss, roc_auc_score

split_idx = int(0.8 * len(model_df))

X_train = X.iloc[:split_idx]
y_train = y.iloc[:split_idx]
X_valid = X.iloc[split_idx:]
y_valid = y.iloc[split_idx:]

model = XGBClassifier(
    n_estimators=600,
    learning_rate=0.04,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    eval_metric='logloss',
    tree_method='hist',
    random_state=42
)

model.fit(X_train, y_train)

p_train = model.predict_proba(X_train)[:, 1]
p_valid = model.predict_proba(X_valid)[:, 1]

print("Train logloss:", log_loss(y_train, p_train))
print("Valid logloss:", log_loss(y_valid, p_valid))
print("Valid Brier:", brier_score_loss(y_valid, p_valid))
print("Valid AUC:", roc_auc_score(y_valid, p_valid))


Train logloss: 0.3699366772820688
Valid logloss: 0.693539268333252
Valid Brier: 0.2474452511090282
Valid AUC: 0.5926189347241979


Calibrate probabilities

In [31]:
from sklearn.linear_model import LogisticRegression
import numpy as np

eps = 1e-6
p_valid_clip = np.clip(p_valid, eps, 1 - eps)
logit_valid = np.log(p_valid_clip / (1 - p_valid_clip)).reshape(-1, 1)

calib = LogisticRegression()
calib.fit(logit_valid, y_valid)

p_valid_cal = calib.predict_proba(logit_valid)[:, 1]

print("Calibrated Valid logloss:", log_loss(y_valid, p_valid_cal))
print("Calibrated Valid Brier:", brier_score_loss(y_valid, p_valid_cal))


Calibrated Valid logloss: 0.6688344766404829
Calibrated Valid Brier: 0.23822021743983254


Re-run predictions for today with new features

In [32]:
def get_latest_pre_profile(fighter_name):
    sub = fighter_prof[fighter_prof['fighter'] == fighter_name].dropna(subset=['pre_win_mean'])
    if sub.empty:
        return None
    return sub.sort_values('Date').iloc[-1]

def build_features_for_matchup(f1_name, f2_name, pre_cols, feature_cols):
    r1 = get_latest_pre_profile(f1_name)
    r2 = get_latest_pre_profile(f2_name)
    if r1 is None or r2 is None:
        print(f"No history for {f1_name} or {f2_name}")
        return None
    
    feat = {}
    for c in pre_cols:
        base = c[4:]  # strip 'pre_'
        name = 'diff_' + base
        if name in feature_cols:
            feat[name] = r1[c] - r2[c]
    
    # add elo_diff using final Elo ratings
    r_elo1 = elo.get(f1_name, START_RATING)
    r_elo2 = elo.get(f2_name, START_RATING)
    feat['elo_diff'] = r_elo1 - r_elo2
    
    feat_vec = pd.DataFrame([feat])[feature_cols]
    return feat_vec


In [33]:
def predict_win_prob(f1_name, f2_name):
    feat_vec = build_features_for_matchup(f1_name, f2_name, pre_cols, feature_cols)
    if feat_vec is None:
        return None
    
    raw_p = model.predict_proba(feat_vec)[:, 1][0]
    
    # calibrated probability
    raw_p = np.clip(raw_p, eps, 1 - eps)
    logit_p = np.log(raw_p / (1 - raw_p)).reshape(-1, 1)
    p_f1 = calib.predict_proba(logit_p)[:, 1][0]
    return p_f1


In [34]:
today_fights = [
    ("Arman Tsarukyan", "Dan Hooker"),
    ("Ian Machado Garry", "Belal Muhammad"),
    ("Volkan Oezdemir", "Alonzo Menifield"),
    ("Mykytybek Orolbai", "Jack Hermansson"),
    ("Waldo Cortes-Acosta", "Shamil Gaziev"),
    ("Tagir Ulanbekov", "Kyoji Horiguchi"),
]

amer_odds = {
    ("Arman Tsarukyan", "Dan Hooker"):        (-650,  470),
    ("Ian Machado Garry", "Belal Muhammad"):  (-270,  220),
    ("Volkan Oezdemir", "Alonzo Menifield"):  (-225,  185),
    ("Mykytybek Orolbai", "Jack Hermansson"): (-250,  205),
    ("Waldo Cortes-Acosta", "Shamil Gaziev"): (-135,  114),
    ("Tagir Ulanbekov", "Kyoji Horiguchi"):   (-218,  180),
}


In [35]:
def get_latest_pre_profile(fighter_name):
    sub = fighter_prof[fighter_prof['fighter'] == fighter_name].dropna(subset=['pre_win_mean'])
    if sub.empty:
        return None
    return sub.sort_values('Date').iloc[-1]


In [36]:
def build_features_for_matchup(f1_name, f2_name, pre_cols, feature_cols):
    r1 = get_latest_pre_profile(f1_name)
    r2 = get_latest_pre_profile(f2_name)
    if r1 is None or r2 is None:
        print(f"No history for {f1_name} or {f2_name}")
        return None
    
    feat = {}
    # diff_ features from pre_ columns (mean + ema)
    for c in pre_cols:
        base = c[4:]            # strip "pre_"
        name = 'diff_' + base
        if name in feature_cols:
            feat[name] = r1[c] - r2[c]
    
    # Elo rating difference
    r_elo1 = elo.get(f1_name, START_RATING)
    r_elo2 = elo.get(f2_name, START_RATING)
    feat['elo_diff'] = r_elo1 - r_elo2
    
    # ensure same column order as training
    feat_vec = pd.DataFrame([feat])[feature_cols]
    return feat_vec


In [37]:
import numpy as np

eps = 1e-6  # safe clip

def predict_win_prob(f1_name, f2_name):
    feat_vec = build_features_for_matchup(f1_name, f2_name, pre_cols, feature_cols)
    if feat_vec is None:
        return None
    
    raw_p = model.predict_proba(feat_vec)[:, 1][0]
    
    # try to apply calibration if 'calib' exists
    try:
        raw_p = np.clip(raw_p, eps, 1 - eps)
        logit_p = np.log(raw_p / (1 - raw_p)).reshape(-1, 1)
        p_f1 = calib.predict_proba(logit_p)[:, 1][0]
    except NameError:
        # no calibration fitted → use raw model prob
        p_f1 = raw_p
    
    return p_f1


In [38]:
def american_to_decimal(o):
    if o > 0:
        return 1 + o / 100.0
    else:
        return 1 + 100.0 / (-o)

def expected_value(p_win, dec_odds, stake=1.0):
    # profit if win: (dec_odds - 1)*stake
    # loss if lose: stake
    return p_win * (dec_odds - 1.0) * stake - (1 - p_win) * stake


In [39]:
results = []

for f1, f2 in today_fights:
    p_f1 = predict_win_prob(f1, f2)
    if p_f1 is None:
        continue
    p_f2 = 1 - p_f1
    
    o1, o2 = amer_odds[(f1, f2)]
    d1 = american_to_decimal(o1)
    d2 = american_to_decimal(o2)
    
    ev1 = expected_value(p_f1, d1)
    ev2 = expected_value(p_f2, d2)
    
    results.append({
        "F1": f1,
        "F2": f2,
        "p_F1_model": p_f1,
        "p_F2_model": p_f2,
        "F1_american": o1,
        "F2_american": o2,
        "F1_decimal": d1,
        "F2_decimal": d2,
        "EV_F1": ev1,
        "EV_F2": ev2,
    })

results_df = pd.DataFrame(results)
results_df


No history for Mykytybek Orolbai or Jack Hermansson
No history for Tagir Ulanbekov or Kyoji Horiguchi


,F1,F2,p_F1_model,p_F2_model,F1_american,F2_american,F1_decimal,F2_decimal,EV_F1,EV_F2
0,Arman Tsarukyan,Dan Hooker,0.692161,0.307839,-650,470,1.153846,5.70,-0.201352,0.754681
1,Ian Machado Garry,Belal Muhammad,0.518026,0.481974,-270,220,1.370370,3.20,-0.290113,0.542317
2,Volkan Oezdemir,Alonzo Menifield,0.595911,0.404089,-225,185,1.444444,2.85,-0.139239,0.151652
3,Waldo Cortes-Acosta,Shamil Gaziev,0.788885,0.211115,-135,114,1.740741,2.14,0.373243,-0.548213


In [40]:
# Best value on favourites
results_df.sort_values("EV_F1", ascending=False)

# Best value on underdogs
results_df.sort_values("EV_F2", ascending=False)


,F1,F2,p_F1_model,p_F2_model,F1_american,F2_american,F1_decimal,F2_decimal,EV_F1,EV_F2
0,Arman Tsarukyan,Dan Hooker,0.692161,0.307839,-650,470,1.153846,5.70,-0.201352,0.754681
1,Ian Machado Garry,Belal Muhammad,0.518026,0.481974,-270,220,1.370370,3.20,-0.290113,0.542317
2,Volkan Oezdemir,Alonzo Menifield,0.595911,0.404089,-225,185,1.444444,2.85,-0.139239,0.151652
3,Waldo Cortes-Acosta,Shamil Gaziev,0.788885,0.211115,-135,114,1.740741,2.14,0.373243,-0.548213
